In [ ]:
import pandas as pd
from google.cloud import storage
import os.path
import os
import gzip
import shutil

NROWS = None

client = storage.Client()
bucket = client.get_bucket('facebook-posts')

for blob in bucket.list_blobs():
    if not blob.name.lower().endswith('.gzip'):
        continue
    path = os.path.join(os.getcwd(), blob.name)
    if not os.path.exists(path):
        print('Downloading {0} to {1} ...'.format(blob.name, path))
        with open(path, 'wb') as file_obj:
            blob.download_to_file(file_obj)
        print('Downloaded {0} to {1}!'.format(blob.name, path))
    
    print('Extracting {0}'.format(path))
    with gzip.open(path, 'rb') as f_in, open(path + ".csv", 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
    print('Extracted {0}'.format(path))

In [ ]:
import pandas as pd
posts = pd.DataFrame()
for i in range(40):
    print('./posts-0000000000{:0>2d}.gzip.csv'.format(i))
    p = pd.read_csv('./posts-{:0>12d}.gzip.csv'.format(i))
    p = p[p['type'] == 'link']
    posts = posts.append(p, ignore_index=True)
    print(posts.shape)

In [ ]:
posts

In [1]:
import pickle
with open('link_posts.pkl', 'wb') as f:
    pickle.dump(posts, f)

PermissionError: [Errno 13] Permission denied: 'link_posts.pkl'

In [2]:
import pickle
with open('link_posts.pkl', 'rb') as f:
    link_posts = pickle.load(f)

In [3]:
link_posts.shape

(2228867, 23)

In [9]:
i_posts = link_posts[link_posts['page_id'].astype(str) == '139188202817559']

In [24]:
i_posts[['id', 'name', 'caption', 'message', 'description']]

,id,name,caption,message,description
379933,139188202817559_1467652189971147,Facebook is Testing a 'Thumbs Down' Dislike Bu...,interestingengineering.com,Some Facebook Messenger users have a new way t...,NaN
384216,139188202817559_1466538313415868,Watch Insect-Inspired Robot Titan-XIII Make It...,interestingengineering.com,The Titan-XIII was used to prove that insect-s...,NaN
389047,139188202817559_1466259526777080,How Engineers Can Market Themselves to Get a Job,interestingengineering.com,We aren't known for our ability to communicate...,NaN
391282,139188202817559_1468353783234321,This Concept BMW Alpha Bullet Bike Is Inspired...,interestingengineering.com,"Designer Mehmet Doruk Erdem and builder Mark ""...",NaN
393328,139188202817559_1466231086779924,These Cool Energy Storage Trains Simply Work W...,interestingengineering.com,Using energy storage trains is a fantastic ide...,NaN
407013,139188202817559_1463364583733241,What Facebook Knows About You and the Algorith...,interestingengineering.com,Ever wonder how you see an ad on Facebook abou...,NaN
409681,139188202817559_1462841403785559,Boston Dynamics Unveils New 6.5 Foot Tall Robo...,interestingengineering.com,"The Boston Dynamics team is back at it again, ...",NaN
414501,139188202817559_1467029020033464,The Real Reason Why the Supersonic Passenger J...,interestingengineering.com,"Why did Concorde, one of the greatest aircraft...",NaN
415978,139188202817559_1465245516878481,Watch the World's Fastest Nerf Dart Go Right T...,interestingengineering.com,400 PSI Nerf cannon made out of a steel tube i...,NaN
418711,139188202817559_1464457510290615,WATCH : DARPA Tests Portable Robot Arm That Ca...,interestingengineering.com,DARPA has found a way to safely land a drone i...,NaN


In [27]:
import TextPreprocess.Language.detector as ld
link_posts['lang_name'] = link_posts['name'].astype(str).apply(ld.detect)
#posts['lang_description'] = posts['description'].astype(str).apply(ld.detect)
#posts['lang_about'] = posts['about'].astype(str).apply(ld.detect)

#%timeit pages['name'].apply(lambda x: ld.detect(x, 'langdetect'))
#%timeit pages['about'].astype(str).apply(lambda x: ld.detect(x))
#%timeit pages['about'].astype(str).apply(lambda x: ld.detect(x, 'langdetect'))

In [28]:
en_link_posts = link_posts[link_posts['lang_name'] == 'en']

In [30]:
en_link_posts.shape

(991673, 24)

In [31]:
import pickle
with open('eng_link_posts.pkl', 'wb') as f:
    pickle.dump(en_link_posts, f)

PermissionError: [Errno 13] Permission denied: 'eng_link_posts.pkl'

In [59]:
import pickle
with open('eng_link_posts.pkl', 'rb') as f:
    en_link_posts = pickle.load(f)

In [60]:
en_link_posts.shape

(991673, 24)

In [63]:
en_link_posts[en_link_posts['page_id'].astype(str) == '503159409740331']

,name,caption,message,description,status_type,type,link,full_picture,picture,source,...,share,comment,like,reaction,love,haha,wow,sad,angry,lang_name


In [ ]:
import Utils.GoogleCloud.storage as gcs
gcs.upload('facebook-posts-processed', 'eng_link_posts.pkl', 'eng_link_posts.pkl')

In [37]:
import Utils.GoogleCloud.storage as gcs
gcs.download('facebook-pages', 'pages.csv', './pages.csv')

done



In [38]:
import pandas as pd

NROWS = None

pages = pd.read_csv('./pages.csv', nrows=NROWS)
pages = pages[['id', 'fan_count']]

In [39]:
result = pd.merge(en_link_posts, pages, left_on='page_id', right_on='id')

In [40]:
result['share_ratio'] = result['share'] / result['fan_count']

In [41]:
result['normalized_share_ratio'] = result['share_ratio'] / result['share_ratio'].max()

In [42]:
result['normalized_share_ratio'].describe()

count    990127.000000
mean          0.000442
std           0.002739
min           0.000000
25%           0.000005
50%           0.000037
75%           0.000190
max           1.000000
Name: normalized_share_ratio, dtype: float64

In [43]:
data = result[['name', 'normalized_share_ratio']]

In [ ]:
data['name'] = data['name'].astype(str)
data['name']

In [ ]:
import nltk
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> stopword
    Error loading stopword: Package 'stopword' not found in index

---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> stopwords
Command 'stopwords' unrecognized

---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> stopwords
      Unzipping corpora/stopw

In [ ]:
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
import re

def string_to_words( string ):
    # Function to convert a raw review to a string of words
    # The input is a single string (a raw movie review), and 
    # the output is a single string (a preprocessed movie review)
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(string).get_text() 
    #
    # 2. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    #
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()                             
    #
    # 4. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    stops = set(stopwords.words("english"))                  
    # 
    # 5. Remove stop words
    meaningful_words = [w for w in words if not w in stops]   
    #
    # 6. Join the words back into one string separated by space, 
    # and return the result.
    return( " ".join( meaningful_words ))   

In [ ]:
# Get the number of reviews based on the dataframe column size
num_name = data["name"].size

# Initialize an empty list to hold the clean reviews
clean_train_name = []

# Loop over each review; create an index i that goes from 0 to the length
# of the movie review list 
for i in range( 0, num_name ):
    # Call our function for each one, and add the result to the list of
    # clean reviews
    if( (i+1)%1000 == 0 ):
        print("Review %d of %d\n" % ( i+1, num_name ))
    clean_train_name.append( string_to_words( data["name"][i] ) )

In [ ]:
import pickle
with open('clean_list_post_name.pkl', 'wb') as f:
    pickle.dump(clean_train_name, f)

In [ ]:
import pickle
with open('clean_list_post_name.pkl', 'rb') as f:
    clean_train_name = pickle.load(f)

In [ ]:
import Utils.GoogleCloud.storage as gcs
gcs.upload('facebook-posts-processed', 'clean_list_post_name.pkl', 'clean_list_post_name.pkl')



In [ ]:
print("Creating the bag of words...\n")
from sklearn.feature_extraction.text import CountVectorizer

# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.  
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000) 

# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of 
# strings.
train_data_features = vectorizer.fit_transform(clean_train_name)

# Numpy arrays are easy to work with, so convert the result to an 
# array
train_data_features = train_data_features.toarray()

In [ ]:
# Take a look at the words in the vocabulary
vocab = vectorizer.get_feature_names()
print(vocab)

In [ ]:
import pickle
with open('clean_list_post_name_features.pkl', 'wb') as f:
    pickle.dump(train_data_features, f)

In [ ]:
print("Training the random forest...")
from sklearn.ensemble import RandomForestRegressor

# Initialize a Random Forest classifier with 100 trees
forest = RandomForestRegressor(n_estimators = 1, n_jobs=-1, verbose = 1, max_depth=3) 

# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run
forest = forest.fit( train_data_features, data["normalized_share_ratio"] )

In [ ]:
data["normalized_share_ratio"].describe()

In [ ]:
data["good"] = data["normalized_share_ratio"].apply(lambda x: 1 if x >= 0.000190 else 0)

In [ ]:
data["good"] = data["good"].astype(int)
data.describe()

In [ ]:
print("Training the random forest...")
from sklearn.ensemble import RandomForestClassifier

# Initialize a Random Forest classifier with 100 trees
forest = RandomForestClassifier(n_estimators = 1000, n_jobs=-1, verbose = 1, max_depth=5) 

# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run
forest = forest.fit( train_data_features, data["good"] )

In [ ]:
forest.feature_importances_

In [ ]:
importances = list(zip(vocab, forest.feature_importances_))
importances.sort(key = lambda t: t[1], reverse=True)
importances

In [ ]:
train_data_features = train_data_features.astype('uint8')

In [ ]:
from sklearn import linear_model
clf = linear_model.Lasso(alpha=0.1, max_iter=10, copy_X =False)
clf.fit(train_data_features, data["normalized_share_ratio"])
print(clf.coef_)

In [ ]:
import xgboost as xgb

gbm = xgb.XGBClassifier(max_depth=3, n_estimators=100).fit(train_data_features, data["normalized_share_ratio"])
gbm

In [58]:
en_link_posts.shape

(991673, 24)

In [57]:
en_link_posts[en_link_posts['page_id'].astype(str)=='503159409740331']

,name,caption,message,description,status_type,type,link,full_picture,picture,source,...,share,comment,like,reaction,love,haha,wow,sad,angry,lang_name


In [51]:
en_link_posts['page_id'].astype(str).unique().tolist()

['28488837485',
 '48243460844',
 '76778137323',
 '789439407744270',
 '236978329705763',
 '6636341311',
 '228735667216',
 '117112876419',
 '103368083911',
 '27566611155',
 '132304361460',
 '129687863720149',
 '134486075205',
 '18512903240',
 '61511025427',
 '153924901307555',
 '248183641886114',
 '512435842131215',
 '221013509347',
 '262059950576033',
 '121111248052564',
 '8908243810',
 '61539048747',
 '228585213818644',
 '10664530778',
 '32536990172',
 '157062725807',
 '137309777563',
 '442386295797314',
 '212009668822281',
 '131247023568988',
 '6070248875',
 '109161584803',
 '211233702271319',
 '97212224368',
 '1422872221315213',
 '134570403224681',
 '340327135979921',
 '835343619870055',
 '45988978793',
 '112671931859',
 '59306617060',
 '148757037599',
 '143022919069931',
 '123159412963',
 '33796084659',
 '7585969235',
 '114186148600459',
 '19440638720',
 '218246874862570',
 '7720276612',
 '1765033567057615',
 '147307165256',
 '19457049254',
 '92979593115',
 '123185754376465',
 '1900

In [52]:
page_ids = en_link_posts['page_id'].astype(str).unique().tolist()

In [55]:
len(page_ids)

55989